In [1]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import urllib
import itertools
import random, os, glob
from imutils import paths
from sklearn.utils import shuffle
from urllib.request import urlopen

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import  ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

2023-02-25 21:02:53.876492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
target_size = (224, 224)
waste_labels = {"Keyboard":0, "Mouse":1, "Computers":2}
dir_path = "./Electronics"

In [5]:
def load_dataset(path):
  x = []
  labels = []
  image_paths = sorted(list(paths.list_images(path)))
  for image_path in image_paths:
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    x.append(img)
    label = image_path.split(os.path.sep)[-2]
    labels.append(waste_labels[label])
  x, labels = shuffle(x, labels, random_state=42)
  input_shape = (np.array(x[0]).shape[1], np.array(x[0]).shape[1], 3)
  print("X shape: ", np.array(x).shape)
  print(f"Number of Labels: {len(np.unique(labels))} , Number of Observation: {len(labels)}")
  print("Input Shape: ", input_shape)
  return x, labels, input_shape

In [6]:
x, labels, input_shape = load_dataset(dir_path)

X shape:  (115, 224, 224, 3)
Number of Labels: 3 , Number of Observation: 115
Input Shape:  (224, 224, 3)


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (224, 224, 3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(3,activation = "softmax")   #Adding the Output Layer
])

2023-02-25 21:17:34.848854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [9]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(lr=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [10]:
bs=30         #Setting batch size
train_dir = "./Electronics/Train"   #Setting training directory
validation_dir = "./Electronics/Valid"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(224,224))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(224,224))

Found 96 images belonging to 3 classes.
Found 19 images belonging to 3 classes.


In [14]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=50 // bs,
                    epochs=20,
                    validation_steps=50 // bs,
                    verbose=2)

Epoch 1/20
1/1 - 6s - loss: 1.2791 - acc: 0.3333 - val_loss: 0.8473 - val_acc: 0.6316 - 6s/epoch - 6s/step
Epoch 2/20
1/1 - 4s - loss: 0.4794 - acc: 1.0000 - val_loss: 0.8883 - val_acc: 0.6316 - 4s/epoch - 4s/step
Epoch 3/20
1/1 - 9s - loss: 0.9259 - acc: 0.5333 - val_loss: 0.8916 - val_acc: 0.6316 - 9s/epoch - 9s/step
Epoch 4/20
1/1 - 4s - loss: 0.7825 - acc: 0.5000 - val_loss: 0.8550 - val_acc: 0.6316 - 4s/epoch - 4s/step
Epoch 5/20
1/1 - 5s - loss: 0.7725 - acc: 0.8333 - val_loss: 0.7800 - val_acc: 0.6316 - 5s/epoch - 5s/step
Epoch 6/20
1/1 - 9s - loss: 0.8898 - acc: 0.6333 - val_loss: 0.7281 - val_acc: 0.6316 - 9s/epoch - 9s/step
Epoch 7/20
1/1 - 4s - loss: 0.7467 - acc: 0.5000 - val_loss: 0.7017 - val_acc: 0.6316 - 4s/epoch - 4s/step
Epoch 8/20
1/1 - 9s - loss: 0.7735 - acc: 0.7000 - val_loss: 0.6680 - val_acc: 0.6316 - 9s/epoch - 9s/step
Epoch 9/20
1/1 - 4s - loss: 0.3062 - acc: 0.8333 - val_loss: 0.6665 - val_acc: 0.6316 - 4s/epoch - 4s/step
Epoch 10/20
1/1 - 8s - loss: 0.4613 -

In [15]:
model.save('./electronicmodel.h5')

In [16]:
def CNN_model_testing(path):
  img = image.load_img(path, target_size=(target_size))
  img = image.img_to_array(img, dtype=np.uint8)
  img = np.array(img)/255.0
  p = model.predict(img.reshape(1,224,224,3))
  predicted_class = np.argmax(p[0])
  return img, p, predicted_class

In [20]:
img, p, predicted_class = CNN_model_testing("./rob_computer.jpeg")

1/1 [==============================] - 0s 46ms/step


In [21]:
p

array([[    0.52045,     0.28231,     0.19723]], dtype=float32)